In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building and Deploying a LangGraph Application with Reasoning Engine in Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Freasoning-engine%2Ftutorial_langgraph.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/reasoning-engine/tutorial_langgraph.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_langgraph.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| | |
|-|-|
| Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) is a managed service that helps you to build and deploy an agent reasoning framework. [LangGraph](https://langchain-ai.github.io/langgraph/) is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

This notebook demonstrates how to build, deploy, and test a simple LangGraph application using [Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) in Vertex AI. You'll learn how to combine LangGraph's workflow orchestration with the scalability of Vertex AI, which enables you to build custom generative AI applications.

Note that the approach used in this notebook defines a [custom application template in Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/customize), which can be extended to LangChain or other orchestration frameworks. If just want to use LangChain on Vertex AI to build agentic generative AI applications, refer to the documentation for [developing with the LangChain template in Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop).

This notebook covers the following steps:

- **Define Tools**: Create custom Python functions to act as tools your AI application can use.
- **Define Router**: Set up routing logic to control conversation flow and tool selection.
- **Build a LangGraph Application**: Structure your application using LangGraph, including the Gemini model and custom tools that you define.
- **Local Testing**: Test your LangGraph application locally to ensure functionality.
- **Deploying to Vertex AI**: Seamlessly deploy your LangGraph application to Reasoning Engine for scalable execution.
- **Remote Testing**: Interact with your deployed application through Vertex AI, testing its functionality in a production-like environment.
- **Cleaning Up Resources**: Delete your deployed application on Vertex AI to avoid incurring unnecessary charges.

By the end of this notebook, you'll have the skills and knowledge to build and deploy your own custom generative AI applications using LangGraph, Reasoning Engine, and Vertex AI.

## Get started

### Install Vertex AI SDK and other required packages

In [1]:
%pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    langgraph \
    httpx

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://[your-staging-bucket]"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Building and deploying a LangGraph app on Reasoning Engine

In the following sections, we'll walk through the process of building and deploying a LangGraph application using Reasoning Engine in Vertex AI.

### Import libraries

Import the necessary Python libraries. These libraries provide the tools we need to interact with LangGraph, Vertex AI, and other components of our application.

In [5]:
from typing import Literal

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode
from vertexai.preview import reasoning_engines

### Define tools

You'll start by defining the a tool for your LangGraph application. You'll define a custom Python function that act as tools in our agentic application.

In this case, we'll define a simple tool that returns a product description based on the product that the user asks about. In reality, you can write functions to call APIs, query databases, or anything other tasks that you might want your agent to be able to use.

In [6]:
@tool
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "smartphone": "A cutting-edge smartphone with advanced camera features and lightning-fast processing.",
        "coffee": "A rich, aromatic blend of ethically sourced coffee beans.",
        "shoes": "High-performance running shoes designed for comfort, support, and speed.",
        "headphones": "Wireless headphones with advanced noise cancellation technology for immersive audio.",
        "speaker": "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    }
    return details.get(product_name, "Product details not found.")

### Define router

Then, you'll define a router to control the flow of the conversation, determining which tool to use based on user input or the state of the interaction. Here we'll use a simple router setup, and you can customize the behavior of your router to handle multiple tools, custom logic, or multi-agent workflows.

In [7]:
def router(state: list[BaseMessage]) -> Literal["get_product_details", "__end__"]:
    """Initiates product details retrieval if the user asks for a product."""
    # Get the tool_calls from the last message in the conversation history.
    tool_calls = state[-1].tool_calls
    # If there are any tool_calls
    if len(tool_calls):
        # Return the name of the tool to be called
        return "get_product_details"
    else:
        # End the conversation flow.
        return "__end__"

### Define LangGraph application

Now you'll bring everything together to define your LangGraph application as a custom template in Reasoning Engine.

This application will use the tool and router that you just defined. LangGraph provides a powerful way to structure these interactions and leverage the capabilities of LLMs.

In [8]:
class SimpleLangGraphApp:
    def __init__(self, project: str, location: str) -> None:
        self.project_id = project
        self.location = location

    # The set_up method is used to define application initialization logic
    def set_up(self) -> None:
        model = ChatVertexAI(model="gemini-1.5-pro")

        builder = MessageGraph()

        model_with_tools = model.bind_tools([get_product_details])
        builder.add_node("tools", model_with_tools)

        tool_node = ToolNode([get_product_details])
        builder.add_node("get_product_details", tool_node)
        builder.add_edge("get_product_details", END)

        builder.set_entry_point("tools")
        builder.add_conditional_edges("tools", router)

        self.runnable = builder.compile()

    # The query method will be used to send inputs to the agent
    def query(self, message: str):
        """Query the application.

        Args:
            message: The user message.

        Returns:
            str: The LLM response.
        """
        chat_history = self.runnable.invoke(HumanMessage(message))

        return chat_history[-1].content

### Local testing

In this section, you'll test your LangGraph app locally before deploying it to ensure that it behaves as expected before deployment.

In [9]:
agent = SimpleLangGraphApp(project=PROJECT_ID, location=LOCATION)
agent.set_up()

In [10]:
agent.query(message="Get product details for shoes")

'High-performance running shoes designed for comfort, support, and speed.'

In [11]:
agent.query(message="Get product details for coffee")

'A rich, aromatic blend of ethically sourced coffee beans.'

In [12]:
agent.query(message="Get product details for smartphone")

'A cutting-edge smartphone with advanced camera features and lightning-fast processing.'

In [13]:
# Ask a question that cannot be answered using the defined tools
agent.query(message="Tell me about the weather")

"I need more information about what you'd like to know about the weather. For example, I can tell you the weather in a specific location if you provide me with the location. \n"

### Deploy your LangGraph app

Now that you verified that your LangGraph application is working locally, it's time to deploy it to Reasoning Engine! This will make your application accessible remotely and allow you to integrate it into larger systems or provide it as a service.

In [14]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    SimpleLangGraphApp(project=PROJECT_ID, location=LOCATION),
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "langgraph",
        "httpx",
    ],
    display_name="Reasoning Engine with LangGraph",
    description="This is a sample custom application in Reasoning Engine that uses LangGraph",
    extra_packages=[],
)

Using bucket your-bucket-name
Writing to gs://your-bucket-name/reasoning_engine/reasoning_engine.pkl
Writing to gs://your-bucket-name/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://your-bucket-name/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/your-project-number/locations/us-central1/reasoningEngines/8081041028246667264/operations/701841121235435520
ReasoningEngine created. Resource name: projects/your-project-number/locations/us-central1/reasoningEngines/8081041028246667264
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/your-project-number/locations/us-central1/reasoningEngines/8081041028246667264')


### Remote test

Now that your LangGraph app is running on Reasoning Engine, let's test it out by querying it in the remote environment:

In [15]:
remote_agent.query(message="Get product details for shoes")

'High-performance running shoes designed for comfort, support, and speed.'

In [16]:
remote_agent.query(message="Get product details for coffee")

'A rich, aromatic blend of ethically sourced coffee beans.'

In [17]:
remote_agent.query(message="Get product details for smartphone")

'A cutting-edge smartphone with advanced camera features and lightning-fast processing.'

In [18]:
remote_agent.query(message="Tell me about the weather")

"I'm sorry, I can't provide information about the weather. The available tools lack that functionality. \n"

## Cleaning up

After you've finished experimenting, it's a good practice to clean up your cloud resources. You can delete the deployed Reasoning Engine instance to avoid any unexpected charges on your Google Cloud account.

In [19]:
remote_agent.delete()

Deleting ReasoningEngine : projects/your-project-number/locations/us-central1/reasoningEngines/8081041028246667264
ReasoningEngine deleted. . Resource name: projects/your-project-number/locations/us-central1/reasoningEngines/8081041028246667264
Deleting ReasoningEngine resource: projects/your-project-number/locations/us-central1/reasoningEngines/8081041028246667264
Delete ReasoningEngine backing LRO: projects/your-project-number/locations/us-central1/operations/5119309405732470784
ReasoningEngine resource projects/your-project-number/locations/us-central1/reasoningEngines/8081041028246667264 deleted.
